In [1]:
!pip3 install tensorflow numpy faiss-cpu pillow opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.7 MB/s eta 0:00:00:00:0100:01


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import cv2
import faiss
import glob
from PIL import UnidentifiedImageError

2025-04-19 12:55:07.427758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745067307.899582      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745067308.061301      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress warnings

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Use MirroredStrategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

Number of devices: 2


I0000 00:00:1745067327.744593      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745067327.745296      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [4]:
base_model = ResNet50V2(weights='imagenet', include_top=False, pooling='avg')

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
def extract_features(file_path):
    try:
        img = load_img(file_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_expanded = np.expand_dims(img_array, axis=0)
        img_processed = preprocess_input(img_expanded)
        features = base_model.predict(img_processed)
        return features.flatten()
    except UnidentifiedImageError:
        print(f"Skipping invalid image: {file_path}")

In [6]:
image_path = glob.glob("/kaggle/input/pokemondata/PokemonData/*/*.*")

embeddings = []
for path in image_path:
    features = extract_features(path)
    if features is not None:
        embeddings.append(features)

embeddings = np.array(embeddings)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

I0000 00:00:1745067337.343963      95 service.cc:148] XLA service 0x7bbee00041e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745067337.345770      95 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745067337.345788      95 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745067337.984167      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


I0000 00:00:1745067341.674877      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [9]:
query_image_path = "/kaggle/input/pokemondata/PokemonData/Arbok/2382e5e913f94dd7845e6b1ac733ef18.jpg"

embed = extract_features(query_image_path)
embed = np.array(embed).reshape(1, -1)

distances, indices = index.search(embed, 25)

print("Similar images:")
for id in indices[0]:
    print(image_path[id])
    # print(distances[0][id])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Similar images:
/kaggle/input/pokemondata/PokemonData/Dragonair/1d24bc51b4c74fbd8e83a775fe32986d.jpg
/kaggle/input/pokemondata/PokemonData/Oddish/1b25533602de4e6e8f991799883e1633.jpg
/kaggle/input/pokemondata/PokemonData/Poliwag/bacee21dad164ef98b4ac27c0affd6f2.jpg
/kaggle/input/pokemondata/PokemonData/Venonat/8b70c1416a594169a3c5e34d6f1a4b69.jpg
/kaggle/input/pokemondata/PokemonData/Growlithe/52a08c668fe84c568edf588031b77970.jpg
/kaggle/input/pokemondata/PokemonData/Caterpie/c7056958ac114249825249a0d817d6fe.jpg
/kaggle/input/pokemondata/PokemonData/Blastoise/560d8e7479f243efbbc93c6afe0a3c29.jpg
/kaggle/input/pokemondata/PokemonData/Magikarp/39a19a5d72034672af38667a70ec1f53.jpg
/kaggle/input/pokemondata/PokemonData/Victreebel/1f2a28a23dd7434a83315235456325f1.jpg
/kaggle/input/pokemondata/PokemonData/Bellsprout/14120da7934a4adcb1565f23183ced3a.jpg
/kaggle/input/pokemondata/PokemonData/Porygon/9d7f03bf3570441f889ea14e0358adaf.jpg
/kaggle/input/pokemo